<a href="https://colab.research.google.com/github/jaipandotco2525/python/blob/master/Cheminfo_rdkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chem_info Project Based Learning 2019/09/21
# chem_project team2


### RDKIt

[RDKit]（https://www.rdkit.org/） は、非常に強力で便利なツールキットですが、Google Colaboratoryにはインストールされていないオープンソースの化学情報学ソフトウェアです。

したがって、最初にrdkitパッケージをインストールする必要があります。 Google Colabはすべてのセッションで初期化される仮想サーバーであるため、すべてのセッションをインストールする必要あり。



In [0]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

環境設定

In [0]:
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages')

すべてのパッケージが適切にインストールされると、次のセルに次のテキストが表示されます。

ドキュメントについては、ドキュメントツリーのChem / index.htmlを参照

In [0]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem
# from rdkit import Chem

print(Chem.__doc__)

さらにツールをimport

In [0]:
from numpy import vectorize as vec
import scipy as sp
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors,PandasTools


### Tox21 data set
毒物データベース
今回は7534種類の分子から学習データを75%（このうち活性アリが218種類、活性無しが5297種類）
テストデータを25%（活性アリ78種類、活性無しが1761種類）を準備して重複データを削除して機械学習を行った。

Download a datafile `na-ar.sdf` from the link below.

https://drive.google.com/uc?id=1ALUNDgAmpauEa0Oa6-6TOPAG_uDTy0HH&export=download

Download a datafile `washed-nr-er.tsv` from the link below.

https://docs.google.com/uc?id=1N-Qykm94hz8NFPppYgTYcQwiRhqJNimw&export=download

次のセルを実行します。アップロードするファイルを選択するように求められるので、ローカルマシンからダウンロードしたファイルをアップロードします。

In [0]:
from google.colab import files
uploaded = files.upload()

次のセルを実行します。アップロードするファイルを選択するように求められるので、ローカルマシンからダウンロードしたファイルをアップロードします。

In [0]:
! ls -la

次に、アップロードされた分子データを読み取ります。

（ファイルの読み取り時のエラーを無視します）

 `na-ar.sdf`は元のデータセットです。

`washed-nr-er.tsv`はキュレーションされたデータセットです。

In [0]:
mols=PandasTools.LoadSDF('nr-ar.sdf',smilesName='SMILES',molColName='Molecule',includeFingerprints=True)


In [0]:
mols_washed = pd.read_csv('washed-nr-er.tsv')


新しいデータを読み込むときは詳細なデータ構造を確認

In [0]:
mols.shape

mols_washed.shape

In [0]:
mols.columns

`head()` は大きなデータを確認するのに適した方法です



したがって、何らかの機械学習モデルを使用して、分子構造から毒性を「予測」できると便利です。危険な化学物質を避け、新薬の候補を見つけるのに役立ちます。

ただし、分子構造は非常に複雑なので、分子記述子と呼ばれる数値表現に変換します。


In [0]:
mols[['Active', 'DSSTox_CID', 'FW', 'Formula', 'ID', 'Molecule', 'SMILES']].head(2)

In [0]:
mols['Active'][0:4]

In [0]:
print("Number of   active molecules:  ", list(mols.Active).count('1'))  #Active
print("Number of inactive molecules:  ", list(mols.Active).count('0'))  #Inactive

In [0]:
names = [x[0] for x in Descriptors._descList]
print("Number of descriptors in the rdkit: ", len(names))
np.array(names)[0:10]


In [0]:
for desc in ['TPSA','MaxPartialCharge','SlogP_VSA1','EState_VSA1','SMR_VSA1','MolLogP','MolMR','BalabanJ','Ipc','HallKierAlpha','Kappa1','Kappa2','Kappa3','RingCount','NumHAcceptors','NumHDonors']:
    exec("mols[desc]=vec(Descriptors.{})(mols['Molecule'])".format(desc))
print("shape of data : {}".format(mols.shape))


In [0]:
print(mols.isnull().any())

分子記述子の一覧（それぞれ環構造の特徴を表しているものや、分子分極率等分子の構成に関わる記述子など様々なものがある。）

In [0]:
mols_desc=mols.drop(["MaxPartialCharge","Ipc","Kappa3", "Active", "DSSTox_CID", "FW", "Formula", "ID", "Molecule", "SMILES"], axis=1)

いくつかの分子特性の分布と相関関係を見てみる

In [0]:
def set_color(L):
  tmp = []
  for l in L:
    if l == '1':
      tmp.append("red")
    else:
      tmp.append("palegreen")
  return tmp

pd.plotting.scatter_matrix(mols_desc,figsize=(16,16), hist_kwds={'bins':15},  
                           marker='+', s=8, alpha=.5, c=set_color(mols.Active))
plt.show()

### Classifier

例として、ランダムフォレストを用いて、分子が有毒かどうかを分類してみる。


検証のために、トレーニングデータとテストデータを分離する。

In [0]:
X_train, X_test, y_train, y_test = train_test_split(mols_desc, mols.Active, train_size=0.75, test_size=0.25)

In [0]:
print("Training Data")
print("Number of   active molecules:  ", list(y_train).count('1'))
print("Number of inactive molecules:  ", list(y_train).count('0'))
print("Test Data")
print("Number of   active molecules:  ", list(y_test).count('1'))
print("Number of inactive molecules:  ", list(y_test).count('0'))

### 試した学習手法
•	Random-forest
データセットの様々なサブサンプルにいくつかの決定木分類器を当てはめ、予測精度を向上させて過剰適合を制御するために平均化を使用する手法。

•	Extremely Randomized Trees
Random-Forest は 決定木を作る際に分岐させる位置を網羅的に探索し、基準を決めて最も良い部分を選びますが、この手法では分岐関数をランダムで選択する手法となっております。

•	Gradient Boosting（勾配ブースティング）
決定木を逐次的に増やしていき、１つ前の決定木の誤りを次の決定木が修正するようにして、新たな決定木を順番に生成していく手法。

LightGBM：左右均等に決定木を成⻑させるのではなく、あくまで目的関数を最小にする向きに成⻑させる特徴がある勾配ブースティングの1種の手法

訓練データの特徴量を階級に分けてヒストグラム化することで、意図的に厳密な枝分かれを探さず大規模なデータセットに対して、計算コストを抑えることが可能な手法となっております。


###ここでは、ランダムフォレストで学習させた結果とコードを提示する



In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)


In [0]:
print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:     {:.3f}".format(model.score(X_test , y_test)))
df = pd.DataFrame(model.feature_importances_, index = X_train.columns)
df

scikit-learnライブラリには、何百ものツールとアルゴリズムが実装されています。 化学記述子を使用して分子を予測/分類する他のモデルと方法を選択できます。

